In [1]:
#Antes de iniciar instalar
#pip3 install pydotplus
#pip3 install sklearn
#pip3 install pandas
#pip3 install numpy
#pip3 install matplotlib

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.preprocessing import StandardScaler
from sklearn.externals.six import StringIO  
import numpy as np
from IPython.display import Image  
import os
from joblib import dump, load 
import pydotplus
from io import StringIO
import random as rd
#For cross validation techniques
from sklearn import datasets, metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

/home/zirei/.local/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [3]:
def prediction_example(clf):
    # Each parameter has a range based on "datosLimpios.csv" training file
    # Ranges for simulation may vary if a new Training file is created
    Hour = rd.randrange(0,24)
    Day = rd.randrange(1,32)
    Reach_procentaje = rd.randrange(0,101)
    IsReshare = rd.randrange(0,2)
    RetweetCount = rd.randrange(0,26128)
    # Make new observation
    observation = [[Hour, Day, Reach_procentaje, IsReshare, RetweetCount]] 
    # Generate random values
    print("The generated values are: ", observation)
    # Predict observation's class (STARTING FROM 0 to 100%)    
    prediction = clf.predict(observation)
    print("class Klout predicted is:", prediction[0]) 

In [4]:
def prediction_example2(clf):
    # Each parameter has a range based on "datosLimpios.csv" training file
    # Ranges for simulation may vary if a new Training file is created
    Hour = rd.randrange(0,24)
    Day = rd.randrange(1,32)
    IsReshare = rd.randrange(0,2)
    Reach = rd.randrange(0, 10342453)
    RetweetCount = rd.randrange(0,26128)
    # Make new observation
    observation = [[Hour, Day, IsReshare, Reach, RetweetCount]] 
    # Generate random values
    print("The generated values are: ", observation)
    # Predict observation's class (STARTING FROM 0 to 100%)    
    prediction = clf.predict(observation)
    print("You've got:", prediction[0], "Likes","\n") 

In [5]:
def train_model():
    classifier_filename = 'classifier_model.joblib'
    feature_cols = ['Hour', 'Day', 'Reach_procentaje', 'IsReshare', 'RetweetCount']

    # Check if model file (.joblib extension) exist and use it as the classifier model
    model_exist = os.path.isfile(classifier_filename)

    if model_exist == False:  # Proceed to training
        col_names = ['Hour', 'Day', 'Reach_procentaje', 'IsReshare', 'RetweetCount', 'Klout']
        
        training = pd.read_csv("../datasets/datosLimpios.csv", header=0, names=col_names)  # load dataset
        
        # Split dataset in features and target variable
        x = training[feature_cols] # Features
        y = training.Klout # Target variable
        
        # Split dataset into training set and test set (70% training and 20% test)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
       
        
        clf = DecisionTreeClassifier() # Create Decision Tree classifer object
        clf = clf.fit(x_train,y_train) # Train Decision Tree Classifer
        
        #Accurracy
        y_pred = clf.predict(x_test)
        #Validatión
        score = clf.score(x_train,y_train)
        preds = clf.predict(x_test)
    
        #Metrics
        calc_accuracy(clf, y_test, y_pred)
        validation(clf, score, preds, y_test, x_train, y_train)
                   
        # Graph tree data (the output is a "tree.png" image)
        #graph_data(clf, feature_cols)

        dump(clf, classifier_filename)  # Write model in a file (persistance)
    else:
        clf = load(classifier_filename)  # Load classification model from file 
        print("Running Klout classifier")
        
        # Call prediction_example function (classifying events instead of simulating them with the method
        # "output_events_randomly")
        for _ in range(10):
           prediction_example(clf)

In [6]:
def train_model2():
    classifier_filename = 'classifier_model2.joblib'
    feature_cols = ['Hour', 'Day', 'IsReshare', 'Reach', 'RetweetCount']

    # Check if model file (.joblib extension) exist and use it as the classifier model
    model_exist = os.path.isfile(classifier_filename)

    if model_exist == False:  # Proceed to training
        col_names = ['Hour', 'Day', 'IsReshare', 'Reach', 'RetweetCount', 'Likes']
        
        training = pd.read_csv("../datasets/datosLimpios2.csv", header=0, names=col_names)  # load dataset
        
        # Split dataset in features and target variable
        x = training[feature_cols] # Features
        y = training.Likes # Target variable
        
        # Split dataset into training set and test set (70% training and 20% test)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
       
        
        clf = DecisionTreeClassifier() # Create Decision Tree classifer object
        clf = clf.fit(x_train,y_train) # Train Decision Tree Classifer
        
        #Accurracy
        y_pred = clf.predict(x_test)
        #Validatión
        score = clf.score(x_train,y_train)
        preds = clf.predict(x_test)
    
        #Metrics
        calc_accuracy(clf, y_test, y_pred)
        validation(clf, score, preds, y_test, x_train, y_train)
                   
        # Graph tree data (the output is a "tree.png" image)
        #graph_data(clf, feature_cols)

        dump(clf, classifier_filename)  # Write model in a file (persistance)
    else:
        clf = load(classifier_filename)  # Load classification model from file 
        print("Running Likes classifier")
        
        # Call prediction_example function (classifying events instead of simulating them with the method
        # "output_events_randomly")
        for _ in range(10):
           prediction_example2(clf)

In [7]:
def validation(clf, score, preds, y_test, x_train, y_train):
    # Calculate Accuracy. How often is the classifier correct?
        #Starting model validation metrics
                
        # To apply validation techniques
        kf = KFold(n_splits=5)
        
        print("Metrica del modelo", score)

        scores = cross_val_score(clf, x_train, y_train, cv=kf, scoring="accuracy")

        print("Metricas cross_validation", scores)

        print("Media de cross_validation", scores.mean())

        score_pred = metrics.accuracy_score(y_test, preds)

        print("Metrica en Test", score_pred)
        # End of model metrics

In [8]:
def calc_accuracy(clf, y_test, y_pred):
    # Calculate Accuracy. How often is the classifier correct?
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    #print("Precision: ", metrics.precision_score(y_test, y_pred))
    #print("Sensibilidad: ", metrics.recall_score(y_test, y_pred))
    #print("Puntaje F1: ", metrics.f1_score(y_test, y_pred))

In [9]:
#def graph_data(clf, feature_cols):
    #Creates an image (tree.png) that represents the shape of the classifier_tree
    #This is for illustrative purposes only (it uses a shorter training file than the current training file)
    #dot_data = StringIO()
    #export_graphviz(clf, out_file=dot_data,  
                    #filled=True, rounded=True,
                    #Name of the leaves of the tree
                    #special_characters=True, feature_names = feature_cols, class_names=['0','1','2','3','4','5'])
    #(graph, )= pydotplus.graph_from_dot_data(dot_data.getvalue())
    #graph.write_png('tree.png')
    #Image(graph.create_png())

In [10]:
#Klou statistics
train_model()

Running classifier
The generated values are:  [[15, 23, 61, 1, 22087]]
class Klout predicted is: 95.0
The generated values are:  [[13, 27, 36, 1, 8519]]
class Klout predicted is: 83.0
The generated values are:  [[5, 8, 54, 0, 18406]]
class Klout predicted is: 96.0
The generated values are:  [[22, 7, 36, 0, 25187]]
class Klout predicted is: 83.0
The generated values are:  [[17, 3, 2, 1, 23201]]
class Klout predicted is: 72.0
The generated values are:  [[6, 24, 22, 0, 25566]]
class Klout predicted is: 83.0
The generated values are:  [[3, 28, 46, 1, 7662]]
class Klout predicted is: 96.0
The generated values are:  [[19, 28, 18, 1, 15686]]
class Klout predicted is: 83.0
The generated values are:  [[1, 2, 2, 1, 18312]]
class Klout predicted is: 72.0
The generated values are:  [[6, 15, 14, 0, 24518]]
class Klout predicted is: 47.0


In [31]:
#Likes statistics
train_model2()

Running classifier
The generated values are:  [[10, 13, 1, 437222, 10633]]
You've got: 0.0 Likes 

The generated values are:  [[10, 16, 0, 3688275, 8097]]
You've got: 0.0 Likes 

The generated values are:  [[11, 3, 1, 349565, 10421]]
You've got: 0.0 Likes 

The generated values are:  [[22, 28, 0, 4399417, 7307]]
You've got: 0.0 Likes 

The generated values are:  [[3, 14, 0, 9197677, 4368]]
You've got: 0.0 Likes 

The generated values are:  [[18, 20, 1, 571544, 9902]]
You've got: 0.0 Likes 

The generated values are:  [[5, 16, 0, 4392423, 8354]]
You've got: 0.0 Likes 

The generated values are:  [[15, 6, 0, 8245976, 5428]]
You've got: 0.0 Likes 

The generated values are:  [[3, 14, 0, 5042860, 15794]]
You've got: 0.0 Likes 

The generated values are:  [[15, 20, 0, 7993398, 8611]]
You've got: 0.0 Likes 



In [32]:
#Ejemplos simulados
train_model()
train_model2()

Running classifier
The generated values are:  [[18, 1, 32, 1, 8521]]
class Klout predicted is: 83.0
The generated values are:  [[14, 7, 7, 1, 2298]]
class Klout predicted is: 72.0
The generated values are:  [[21, 5, 3, 0, 6668]]
class Klout predicted is: 70.0
The generated values are:  [[7, 2, 49, 1, 11941]]
class Klout predicted is: 96.0
The generated values are:  [[4, 5, 58, 0, 18474]]
class Klout predicted is: 96.0
The generated values are:  [[23, 31, 19, 1, 20762]]
class Klout predicted is: 83.0
The generated values are:  [[17, 20, 23, 1, 6042]]
class Klout predicted is: 83.0
The generated values are:  [[10, 13, 84, 1, 22687]]
class Klout predicted is: 95.0
The generated values are:  [[6, 18, 20, 0, 13983]]
class Klout predicted is: 83.0
The generated values are:  [[23, 25, 28, 0, 4251]]
class Klout predicted is: 83.0
Running classifier
The generated values are:  [[0, 4, 0, 8834788, 9588]]
You've got: 0.0 Likes 

The generated values are:  [[13, 6, 0, 3735448, 25375]]
You've got: 0